In [1]:
import os

def write_to_file(string, path, filename):
  """This is a helper function to create a file from a string."""
  if not os.path.exists(path) and path != "":
    os.makedirs(path)
  with open(os.path.join(path, filename), "w") as f:
      f.write(string)

In [2]:
path = "."

In [3]:
domain = """
(define (domain open-a-coconut)
    (:requirements :strips :typing)
    (:types
        coconut - item
        hammer - item
        screwdriver - item
        knife - item
        container - item
        towel - item
        player location direction
    )

    (:predicates
        (has_hole ?coconut - coconut) ; whether the coconut has a hole
        (is_open ?coconut - coconut) ; whether the coconut is open
        (has_water ?i - item) ; whether the item has water
        (has_meat ?coconut - item) ; whether the coconut has meat
        (heated ?coconut - coconut) ; whether the coconut is hot
        (in ?i1 - item ?i2 - item) ; whether i1 is in i2
        (on ?i1 - item ?i2 - item) ; whether i1 is on i2
        (ontop ?i - item) ; whether item is on top of something
        (pressed ?i - item) ; whether something is on the item
        (hot ?l - location) ; specifically for oven, true if oven is hot
        (inventory ?p - player ?i - item) ; whether the player has the item in their inventory
        (connected ?l1 - location ?d - direction ?l2 - location) ; whether l1 is connected to l2 via direction d
    )

    (:action move
        :parameters (?p - player ?l1 - location ?d - direction ?l2 - location)
        :precondition (and (at ?p ?l1) (connected ?l1 ?d ?l2))
        :effect (and (not (at ?p ?l1)) (at ?p ?l2))
    )

    (:action get
      :parameters (?i - item ?p - player ?l - location)
      :precondition (and (at ?p ?l) (at ?i ?l) (not (pressed ?i)) (not (ontop ?i)))
      :effect (and (not (at ?i ?l)) (inventory ?p ?i))
    )

    (:action drop
        :parameters (?i - item ?p - player ?l - location)
        :precondition (and (at ?p ?l) (inventory ?p ?i))
        :effect (and (at ?i ?l) (not (inventory ?p ?i)))
    )

    (:action position
        :parameters (?i1 - item ?i2 - item ?p - player ?l - location)
        :precondition (and (at ?p ?l) (at ?i1 ?l) (at ?i2 ?l) (not (pressed ?i1)) (not (pressed ?i2)))
        :effect (and (on ?i1 ?i2) (pressed ?i2) (ontop ?i1)) ; places i1 on top of i2
    )

    (:action remove
        :parameters (?i1 - item ?i2 - item ?p - player ?l - location)
        :precondition (and (at ?p ?l) (at ?i2 ?l) (on ?i1 ?i2) (not (pressed ?i1)))
        :effect (and (not (on ?i1 ?i2)) (not (pressed ?i2)) (not (ontop ?i1))) ; removes i1 from on top of i2
    )

    (:action poke
        :parameters (?i1 - coconut ?i2 - screwdriver ?i3 - hammer ?p - player ?l - location)
        :precondition (and (at ?p ?l) (at ?i1 ?l) (on ?i2 ?i1) (on ?i3 ?i2) (not (pressed ?i3)))
        :effect (and (in ?i2 ?i1) (not (on ?i2 ?i1))) ; pokes hole in coconut with screwdriver, screwdriver now in coconut
    )

    (:action takeout
        :parameters (?i1 - item ?i2 - item ?p - player ?l - location)
        :precondition (and (at ?p ?l) (at ?i1 ?l) (at ?i2 ?l) (in ?i1 ?i2) (not (pressed ?i1)))
        :effect (and (not (in ?i1 ?i2)) (not (pressed ?i2)) (not (ontop ?i1)) (has_hole ?i2)) ; takes screwdriver out of coconut, coconut now has hole
    )

    (:action drain
        :parameters (?i1 - coconut ?i2 - container ?p - player ?l - location)
        :precondition (and (at ?p ?l) (at ?i1 ?l) (at ?i2 ?l) (has_hole ?i1) (has_water ?i1) (not (has_water ?i2)) (on ?i1 ?i2))
        :effect (and (not (has_water ?i1)) (has_water ?i2)) ; drains water from coconut into container
    )

    (:action wrap
        :parameters (?i1 - coconut ?i2 - towel ?p - player ?l - location)
        :precondition (and (at ?p ?l) (at ?i1 ?l) (at ?i2 ?l) (not (pressed ?i1)) (not (pressed ?i2)))
        :effect (and (on ?i2 ?i1) (pressed ?i1) (ontop ?i2)) ; wraps coconut with towel, towel now on top of coconut
    )
    
    (:action strike
        :parameters (?i1 - coconut ?i2 - towel ?i3 - hammer ?p - player ?l - location)
        :precondition (and (at ?p ?l) (at ?i1 ?l) (at ?i2 ?l) (at ?i3 ?l) (on ?i2 ?i1) (on ?i3 ?i2) (not (pressed ?i3)) (not (ontop ?i1)) (not (has_water ?i1)))
        :effect (and (is_open ?i1)) ; strike coconut with hammer, coconut is now open
    )

    (:action smash
        :parameters (?i - coconut ?p - player)
        :precondition (and (at ?p ground) (at ?i ground) (not (is_open ?i)) (not (pressed ?i)) (not (ontop ?i)) (not (has_water ?i)))
        :effect (and (is_open ?i)) ; smashes coconut on ground, coconut is now open
    )

    (:action preheat
        :parameters (?p - player)
        :precondition (and (at ?p oven) (not (hot oven)))
        :effect (and (hot oven)) ; preheats the oven
    )

    (:action bake
        :parameters (?i - coconut ?p - player)
        :precondition (and (at ?p oven) (hot oven) (not (pressed ?i)) (inventory ?p ?i))
        :effect (and (heated ?i) (not (hot oven))) ; bakes the coconut in the oven, oven now cold
    )

    (:action heat
        :parameters (?i - coconut ?p - player)
        :precondition (and (at ?p microwave) (not (pressed ?i)) (inventory ?p ?i))
        :effect (and (heated ?i)) ; heats the coconut in the microwave
    )

    (:action separatemeat
        :parameters (?i1 - coconut ?i2 - knife ?p - player ?l - location)
        :precondition (and (at ?p ?l) (at ?i1 ?l) (at ?i2 ?l) (is_open ?i1) (on ?i2 ?i1) (not (pressed ?i2)) (has_meat ?i1))
        :effect (and (not (has_meat ?i1)) (at meat ?l)) ; separates meat from coconut with knife, place meat at location
    )
)
"""

# Write the domain to a PDDL file
domain_filename = "domain.pddl"
write_to_file(domain, path, domain_filename)

In [4]:
problem1 = """
(define (problem drain-coconut)
    (:domain open-a-coconut)

    (:objects
        self - player
        table - location
        left right up down - direction
        coconut - coconut
        hammer - hammer
        screwdriver - screwdriver
        glass - container
    )

    (:init
        (at coconut table)
        (has_water coconut)
        (at screwdriver table)
        (at hammer table)
        (at glass table)
        (at self table)
    )

    (:goal (and (has_water glass)))
)
"""

# Write the problem to a PDDL file
problem_filename = "1-drain-coconut.pddl"
write_to_file(problem1, path, problem_filename)

In [5]:
# Check your files are formatted correctly
!python -B PDDL.py '{path}/{domain_filename}' '{path}/{problem_filename}'

----------------------------
['define',
 ['domain', 'open-a-coconut'],
 [':requirements', ':strips', ':typing'],
 [':types',
  'coconut',
  '-',
  'item',
  'hammer',
  '-',
  'item',
  'screwdriver',
  '-',
  'item',
  'knife',
  '-',
  'item',
  'container',
  '-',
  'item',
  'towel',
  '-',
  'item',
  'player',
  'location',
  'direction'],
 [':predicates',
  ['has_hole', '?coconut', '-', 'coconut'],
  ['is_open', '?coconut', '-', 'coconut'],
  ['has_water', '?i', '-', 'item'],
  ['has_meat', '?coconut', '-', 'item'],
  ['heated', '?coconut', '-', 'coconut'],
  ['in', '?i1', '-', 'item', '?i2', '-', 'item'],
  ['on', '?i1', '-', 'item', '?i2', '-', 'item'],
  ['ontop', '?i', '-', 'item'],
  ['pressed', '?i', '-', 'item'],
  ['hot', '?l', '-', 'location'],
  ['inventory', '?p', '-', 'player', '?i', '-', 'item'],
  ['connected',
   '?l1',
   '-',
   'location',
   '?d',
   '-',
   'direction',
   '?l2',
   '-',
   'location']],
 [':action',
  'move',
  ':parameters',
  ['?p',
   '-'

In [6]:
# Compute a plan
!python -B planner.py '{path}/{domain_filename}' '{path}/{problem_filename}'

Time: 0.06807899475097656s
plan:
position screwdriver coconut self table
position hammer screwdriver self table
poke coconut screwdriver hammer self table
remove hammer screwdriver self table
takeout screwdriver coconut self table
position coconut glass self table
drain coconut glass self table


In [7]:
problem2 = """
(define (problem crack-coconut)
    (:domain open-a-coconut)

    (:objects
        self - player
        table ground oven microwave - location
        left right up down - direction
        coconut - coconut
        hammer - hammer
        knife - knife
        towel - towel
        meat - item
    )

    (:init
        (connected table down ground)
        (connected ground up table)
        (at coconut table)
        (has_hole coconut)
        (has_meat coconut)
        (at hammer table)
        (at knife table)
        (at towel table)
        (at self table)
    )

    (:goal (and (at meat table)))
)
"""
# Write the problem to a PDDL file
problem_filename = "2-crack-coconut.pddl"
write_to_file(problem2, path, problem_filename)

In [8]:
# Check your files are formatted correctly
!python -B PDDL.py '{path}/{domain_filename}' '{path}/{problem_filename}'

----------------------------
['define',
 ['domain', 'open-a-coconut'],
 [':requirements', ':strips', ':typing'],
 [':types',
  'coconut',
  '-',
  'item',
  'hammer',
  '-',
  'item',
  'screwdriver',
  '-',
  'item',
  'knife',
  '-',
  'item',
  'container',
  '-',
  'item',
  'towel',
  '-',
  'item',
  'player',
  'location',
  'direction'],
 [':predicates',
  ['has_hole', '?coconut', '-', 'coconut'],
  ['is_open', '?coconut', '-', 'coconut'],
  ['has_water', '?i', '-', 'item'],
  ['has_meat', '?coconut', '-', 'item'],
  ['heated', '?coconut', '-', 'coconut'],
  ['in', '?i1', '-', 'item', '?i2', '-', 'item'],
  ['on', '?i1', '-', 'item', '?i2', '-', 'item'],
  ['ontop', '?i', '-', 'item'],
  ['pressed', '?i', '-', 'item'],
  ['hot', '?l', '-', 'location'],
  ['inventory', '?p', '-', 'player', '?i', '-', 'item'],
  ['connected',
   '?l1',
   '-',
   'location',
   '?d',
   '-',
   'direction',
   '?l2',
   '-',
   'location']],
 [':action',
  'move',
  ':parameters',
  ['?p',
   '-'

In [9]:
# Compute a plan
!python -B planner.py '{path}/{domain_filename}' '{path}/{problem_filename}'

Time: 0.5320558547973633s
plan:
position towel coconut self table
position hammer towel self table
strike coconut towel hammer self table
remove hammer towel self table
remove towel coconut self table
position knife coconut self table
separatemeat coconut knife self table


In [10]:
problem3 = """
(define (problem crack-hot-coconut)
    (:domain open-a-coconut)

    (:objects
        self - player
        table ground oven microwave - location
        left right up down - direction
        coconut - coconut
        hammer - hammer
        knife - knife
        towel - towel
        meat - item
    )

    (:init
        (connected table down ground)
        (connected ground up table)
        (connected table right oven)
        (connected oven left table)
        (connected table left microwave)
        (connected microwave right table)
        (at coconut table)
        (has_hole coconut)
        (has_meat coconut)
        (at hammer table)
        (at knife table)
        (at towel table)
        (at self table)
    )

    (:goal (and (at meat table) (heated coconut)))
)
"""
# Write the problem to a PDDL file
problem_filename = "3-crack-coconut.pddl"
write_to_file(problem3, path, problem_filename)

In [11]:
# Check your files are formatted correctly
!python -B PDDL.py '{path}/{domain_filename}' '{path}/{problem_filename}'

----------------------------
['define',
 ['domain', 'open-a-coconut'],
 [':requirements', ':strips', ':typing'],
 [':types',
  'coconut',
  '-',
  'item',
  'hammer',
  '-',
  'item',
  'screwdriver',
  '-',
  'item',
  'knife',
  '-',
  'item',
  'container',
  '-',
  'item',
  'towel',
  '-',
  'item',
  'player',
  'location',
  'direction'],
 [':predicates',
  ['has_hole', '?coconut', '-', 'coconut'],
  ['is_open', '?coconut', '-', 'coconut'],
  ['has_water', '?i', '-', 'item'],
  ['has_meat', '?coconut', '-', 'item'],
  ['heated', '?coconut', '-', 'coconut'],
  ['in', '?i1', '-', 'item', '?i2', '-', 'item'],
  ['on', '?i1', '-', 'item', '?i2', '-', 'item'],
  ['ontop', '?i', '-', 'item'],
  ['pressed', '?i', '-', 'item'],
  ['hot', '?l', '-', 'location'],
  ['inventory', '?p', '-', 'player', '?i', '-', 'item'],
  ['connected',
   '?l1',
   '-',
   'location',
   '?d',
   '-',
   'direction',
   '?l2',
   '-',
   'location']],
 [':action',
  'move',
  ':parameters',
  ['?p',
   '-'

In [12]:
# Compute a plan
!python -B planner.py '{path}/{domain_filename}' '{path}/{problem_filename}'

Time: 9.722341775894165s
plan:
get coconut self table
move self table left microwave
heat coconut self
move self microwave right table
move self table down ground
drop coconut self ground
smash coconut self
get coconut self ground
move self ground up table
drop coconut self table
position knife coconut self table
separatemeat coconut knife self table


In [13]:
problem4 = """
(define (problem full-open-coconut)
    (:domain open-a-coconut)

    (:objects
        self - player
        table ground oven microwave - location
        left right up down - direction
        coconut - coconut
        hammer - hammer
        knife - knife
        towel - towel
        glass - container
        meat - item
    )

    (:init
        (connected table down ground)
        (connected ground up table)
        (connected table right oven)
        (connected oven left table)
        (connected table left microwave)
        (connected microwave right table)
        (at coconut table)
        (has_water coconut)
        (has_meat coconut)
        (at screwdriver table)
        (at hammer table)
        (at knife table)
        (at towel table)
        (at glass table)
        (at self table)
    )

    (:goal (and (has_water glass) (at meat table)))
)
"""
# Write the problem to a PDDL file
problem_filename = "4-open-a-coconut.pddl"
write_to_file(problem4, path, problem_filename)

In [14]:
# Check your files are formatted correctly
!python -B PDDL.py '{path}/{domain_filename}' '{path}/{problem_filename}'

----------------------------
['define',
 ['domain', 'open-a-coconut'],
 [':requirements', ':strips', ':typing'],
 [':types',
  'coconut',
  '-',
  'item',
  'hammer',
  '-',
  'item',
  'screwdriver',
  '-',
  'item',
  'knife',
  '-',
  'item',
  'container',
  '-',
  'item',
  'towel',
  '-',
  'item',
  'player',
  'location',
  'direction'],
 [':predicates',
  ['has_hole', '?coconut', '-', 'coconut'],
  ['is_open', '?coconut', '-', 'coconut'],
  ['has_water', '?i', '-', 'item'],
  ['has_meat', '?coconut', '-', 'item'],
  ['heated', '?coconut', '-', 'coconut'],
  ['in', '?i1', '-', 'item', '?i2', '-', 'item'],
  ['on', '?i1', '-', 'item', '?i2', '-', 'item'],
  ['ontop', '?i', '-', 'item'],
  ['pressed', '?i', '-', 'item'],
  ['hot', '?l', '-', 'location'],
  ['inventory', '?p', '-', 'player', '?i', '-', 'item'],
  ['connected',
   '?l1',
   '-',
   'location',
   '?d',
   '-',
   'direction',
   '?l2',
   '-',
   'location']],
 [':action',
  'move',
  ':parameters',
  ['?p',
   '-'

In [15]:
# Compute a plan
# !python -B planner.py '{path}/{domain_filename}' '{path}/{problem_filename}'